# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.api.types
import math
from scipy.stats import zscore

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor
from lifelines.utils import concordance_index
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from imblearn.under_sampling import RandomUnderSampler

# JCOPML Package
from jcopml.plot import plot_missing_value
from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.tuning import random_search_params as rsp
from jcopml.feature_importance import mean_loss_decrease
from jcopml.feature_importance import mean_score_decrease
from jcopml.tuning import bayes_search_params as bsp
from jcopml.tuning import grid_search_params as gsp

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
data = pd.read_csv("hasil_clustering.csv")
data

,Agency Name,Position Title,YTD Gross Pay,Cluster
0,AGRICULTURE,ACCOUNTANT,55819.64,1
1,AGRICULTURE,ACCOUNTS SUPERVISOR,59947.10,1
2,AGRICULTURE,ACCOUNTS SUPERVISOR,8010.72,1
3,AGRICULTURE,ADMIN SUPPORT ASSISTANT,13880.70,1
4,AGRICULTURE,ADMIN SUPPORT ASSISTANT,10455.10,1
...,...,...,...,...
71502,TRANSPORTATION,TRANSPORTATION PROJECT MGR,94387.00,0
71503,TRANSPORTATION,URBAN TRAFFIC SUPERVISOR,38791.20,0
71504,TRANSPORTATION,URBAN TRAFFIC SUPERVISOR,74517.01,0
71505,TRANSPORTATION,URBAN TRAFFIC SUPERVISOR,83122.81,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
data.select_dtypes(include=["object"]).columns

Index(['Ethnicity', 'Smoking_Status', 'Air_Quality_Index', 'Income_Level'], dtype='object')

In [3]:
# Label Encoding
label_encoder = LabelEncoder()

categorical_columns = data.select_dtypes(include=["object"]).columns
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

data.head()

,Agency Name,Position Title,YTD Gross Pay,Cluster
0,0,4,55819.64,1
1,0,22,59947.10,1
2,0,22,8010.72,1
3,0,39,13880.70,1
4,0,39,10455.10,1


In [4]:
X = data.drop(columns="Cluster")
y = data["Cluster"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((57205, 3), (14302, 3), (57205,), (14302,))

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
X_train.columns

Index(['Agency Name', 'Position Title', 'YTD Gross Pay'], dtype='object')

In [6]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(scaling="minmax"), ['Agency Name', 'Position Title', 'YTD Gross Pay']),
    # ('categoric', cat_pipe(encoder="onehot"), ['Ethnicity', 'Smoking_Status', 'Income_Level'])
])

In [10]:
knn = Pipeline([
    ("prep", preprocessor),
    ("algo", KNeighborsClassifier(n_neighbors=3))
])

print(knn.fit(X_train, y_train))
print(knn.score(X_train, y_train))
print(knn.score(X_val, y_val))

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Agency Name',
                                                   'Position Title',
                                                   'YTD Gross Pay'])])),
                ('algo', KNeighborsClassifier(n_neighbors=3))])
0.99973778515864
0.999160956509579


In [8]:
rf = Pipeline([
    ("prep", preprocessor),
    ("algo", RandomForestClassifier(n_estimators=100))
])

print(rf.fit(X_train, y_train))
print(rf.score(X_train, y_train))
print(rf.score(X_val, y_val))

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Agency Name',
                                                   'Position Title',
                                                   'YTD Gross Pay'])])),
                ('algo', RandomForestClassifier())])
1.0
1.0


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [11]:
print("KNN Evaluation : ")
y_val_pred_knn = knn.predict(X_val)
report_knn = classification_report(y_val, y_val_pred_knn)
print("Classification Report:")
print(report_knn)

cm_knn = confusion_matrix(y_val, y_val_pred_knn)
print("\nConfusion Matrix KNN :")
print(cm_knn)
print("KNN Evaluation :")
print(f"Precision (macro): {precision_score(y_val, y_val_pred_knn, average='macro', zero_division=0):.4f}")
print(f"Recall (macro): {recall_score(y_val, y_val_pred_knn, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_val, y_val_pred_knn, average='macro', zero_division=0):.4f}")


print("\n")
print("Random Forest Evaluation : ")
y_val_pred_rf = rf.predict(X_val)
report_rf = classification_report(y_val, y_val_pred_rf)
print("Classification Report:")
print(report_rf)

cm_rf = confusion_matrix(y_val, y_val_pred_rf)
print("\nConfusion Matrix Random Forest :")
print(cm_rf)
print("\nRandom Forest Evaluation :")
print(f"Precision (macro): {precision_score(y_val, y_val_pred_rf, average='macro', zero_division=0):.4f}")
print(f"Recall (macro): {recall_score(y_val, y_val_pred_rf, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_val, y_val_pred_rf, average='macro', zero_division=0):.4f}")

KNN Evaluation : 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7312
           1       1.00      1.00      1.00      6990

    accuracy                           1.00     14302
   macro avg       1.00      1.00      1.00     14302
weighted avg       1.00      1.00      1.00     14302


Confusion Matrix KNN :
[[7308    4]
 [   8 6982]]
KNN Evaluation :
Precision (macro): 0.9992
Recall (macro): 0.9992
F1-Score (macro): 0.9992


Random Forest Evaluation : 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7312
           1       1.00      1.00      1.00      6990

    accuracy                           1.00     14302
   macro avg       1.00      1.00      1.00     14302
weighted avg       1.00      1.00      1.00     14302


Confusion Matrix Random Forest :
[[7312    0]
 [   0 6990]]

Random Forest Evaluation :
Precision (macro): 1.

| **Model**         | **Accuracy** | **F1-Score** | **Precision** | **Recall** |
|--------------------|--------------|--------------|---------------|------------|
| **KNN**            | 1.00        | 	0.9992    | 	0.9992         | 0.9992      |
| **Random Forest**  | 1.00         | 1.0000        | 1.0000          | 1.0000       |

**Analisis Perbandingan**

**1. Accuracy**
- Random Forest memiliki Accuracy sempurna (1.00), yang menunjukkan bahwa model ini berhasil memprediksi semua sampel dengan benar. Ini adalah hasil yang sangat baik, tetapi perlu diwaspadai kemungkinan overfitting.
- KNN juga memiliki Accuracy sempurna (1.00), yang menunjukkan bahwa model ini juga berhasil memprediksi semua sampel dengan benar. Namun, seperti Random Forest, hasil yang sempurna ini perlu diwaspadai karena kemungkinan overfitting.

**2. F1-Score**
- Random Forest memiliki F1-Score sempurna (1.0000), yang menunjukkan keseimbangan sempurna antara Precision dan Recall. Ini berarti model ini sangat baik dalam memprediksi semua kelas.
- KNN memiliki F1-Score yang sangat tinggi (0.9992), yang menunjukkan performa yang hampir sempurna tetapi sedikit lebih rendah dibandingkan Random Forest.

**3. Precision**
- Random Forest memiliki Precision sempurna (1.0000), yang berarti tidak ada prediksi salah untuk kelas positif.
- KNN memiliki Precision yang sangat tinggi (0.9992), yang menunjukkan bahwa model ini hampir tidak memiliki kesalahan dalam memprediksi kelas positif.

**4. Recall**
- Random Forest memiliki Recall sempurna (1.0000), yang berarti semua instance positif terdeteksi dengan benar.
- KNN memiliki Recall yang sangat tinggi (0.9992), yang menunjukkan bahwa model ini hampir tidak melewatkan instance positif.

**Kesimpulan Perbandingan**
- Random Forest memberikan hasil yang sempurna untuk semua metrik, sehingga dapat dikatakan sebagai model terbaik dalam kasus ini. Namun, hasil yang sempurna ini perlu diwaspadai karena kemungkinan overfitting.
- KNN juga memberikan performa yang hampir sempurna tetapi sedikit lebih rendah dibandingkan Random Forest. Untuk dataset atau skenario lain, model ini mungkin perlu penyesuaian parameter atau metode tambahan untuk meningkatkan performa.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

#### Menggunakan Hyperparameter RandomizedSearchCV

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

**Berdasarkan hasil evaluasi model yang diberikan, berikut adalah analisis kelemahan model, identifikasi overfitting/underfitting, dan rekomendasi tindakan lanjutan :**

**1. Analisis Kelemahan Model**

- **KNN**
    - **Precision: 0.9992**
        - Precision sudah sangat tinggi, tetapi masih ada sedikit ruang untuk meningkatkan kemampuan model dalam meminimalkan prediksi salah pada beberapa kelas.
    - **Recall: 0.9992**
        - Recall mendekati tinggi, tetapi tidak sebaik performa dari Random Forest..
    - **F1-Score: 0.9992**
        - F1-Score menunjukkan bahwa meskipun performa KNN sangat baik, model ini cenderung lebih sensitif terhadap perubahan pada distribusi data dibandingkan model berbasis pohon.

- **Random Forest**
    - **Precision, Recall, dan F1-Score: 1.0000**
        - Model ini tidak menunjukkan kelemahan yang berarti dalam hasil yang diperoleh. Namun, kemungkinan overfitting dapat menjadi perhatian karena akurasi sempurna di dataset validasi mungkin tidak mencerminkan performa yang sama di dataset baru.

**2. Identifikasi Overfitting atau Underfitting**

- **KNN**
    - **Train Score: 0.99973778515864**
    - **Test Score: 0.999160956509579**
    - **Analisis:**
        - Perbedaan antara train score dan test score sangat kecil, yaitu sekitar 0.000577. Ini menunjukkan bahwa model KNN tidak mengalami overfitting yang signifikan.
        - Test score yang sangat tinggi (0.99916) menunjukkan bahwa model KNN mampu menangkap pola data dengan sangat baik dan memiliki kemampuan generalisasi yang baik.
        - Meskipun test score sangat tinggi, jika dataset ini relatif sederhana atau memiliki fitur yang sangat informatif, performa yang hampir sempurna ini mungkin tidak menunjukkan underfitting. Namun, jika dataset lebih kompleks, KNN mungkin kurang mampu menangkap pola yang lebih kompleks, mengindikasikan potensi underfitting.
    - **Kesimpulan: Model KNN tidak menunjukkan overfitting, tetapi ada kemungkinan underfitting jika dataset lebih kompleks daripada yang terlihat.**

- **Random Forest**
    - **Train Score: 1.0**
    - **Test Score: 1.0**
    - **Analisis:**
        - Tidak ada perbedaan antara train score dan test score, yang menunjukkan bahwa model memiliki performa sempurna pada kedua dataset.
        - Dataset yang relatif sederhana atau memiliki fitur yang sangat informatif atau Model Random Forest yang sangat kuat dan mampu mempelajari pola data dengan sempurna..
        - Meskipun tidak ada perbedaan antara train score dan test score, performa sempurna seperti ini jarang terjadi pada dataset dunia nyata yang lebih kompleks dan berisik. Oleh karena itu, perlu diwaspadai kemungkinan overfitting, terutama jika dataset yang digunakan tidak representatif atau terlalu kecil.
    - **Model Random Forest tidak menunjukkan tanda-tanda overfitting atau underfitting pada dataset ini, tetapi perlu diuji pada dataset baru untuk memastikan generalisasi yang baik.**

**3. Rekomendasi Tindakan Lanjutan**

- **Untuk KNN (Kemungkinan Underfitting)**
    -  **Verifikasi Generalisasi :**
        - Uji model pada dataset baru yang belum pernah dilihat selama pelatihan atau pengujian. Ini untuk memastikan bahwa model dapat generalisasi dengan baik ke data baru.
    - **Periksa Kompleksitas Dataset :**
        - ika dataset relatif sederhana, performa yang hampir sempurna mungkin wajar. Namun, jika dataset lebih kompleks, pertimbangkan untuk meningkatkan kompleksitas model KNN (misalnya, dengan menyesuaikan hyperparameter seperti n_neighbors atau menggunakan metrik jarang yang berbeda).
    - **Evaluasi Fitur :**
        - Pastikan fitur yang digunakan sudah optimal. Jika perlu, lakukan feature engineering untuk meningkatkan kualitas fitur.

- **Untuk Random Forest (Tidak Overfitting/Underfitting)**
    - **Verifikasi Generalisasi :**
        - Uji model pada dataset baru yang belum pernah dilihat selama pelatihan atau pengujian. Ini untuk memastikan bahwa model dapat generalisasi dengan baik ke data baru.
    - **Periksa Distribusi Data :**
        - Pastikan bahwa data pelatihan dan data uji memiliki distribusi yang serupa. Jika data uji terlalu mirip dengan data pelatihan, performa sempurna mungkin tidak mencerminkan kemampuan generalisasi yang sebenarnya.
    - **Pertahankan Kompleksitas Model :**
        - Karena model sudah menunjukkan performa yang sangat baik, tidak perlu melakukan perubahan besar. Namun, tetap pantau performa saat diimplementasikan pada data baru.